
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/jkanclerz/data-science-workshop-2024/blob/main/40--spark/02--dataframes.ipynb)

In [ ]:
!apt-get install openjdk-17-jdk-headless -qq > /dev/null
!wget https://dlcdn.apache.org/spark/spark-3.5.4/spark-3.5.4-bin-hadoop3.tgz -O spark-3.5.4-bin-hadoop3.tgz
!tar xf spark-3.5.4-bin-hadoop3.tgz

In [2]:
!pip install -q pyspark

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.4-bin-hadoop3"

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Spark dataframe")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [23]:
sc = spark.sparkContext

In [ ]:
spark

### DataFrames

DataFrames gives a schema view of data basically, it is an abstraction. In dataframes, view of data is organized as columns with column name and types info. In addition, we can say data in dataframe is as same as the table in relational database.

### DataSets

In Spark, datasets are an extension of dataframes. Basically, it earns two different APIs characteristics, such as strongly typed and untyped. Datasets are by default a collection of strongly typed JVM objects

**data frames** allow to deal with data in structured way. We will see how the data frame abstraction, very popular in other data analytics ecosystems (e.g. R and Python/Pandas), it is very powerful when performing exploratory data analysis. In fact, it is very easy to express data queries when used together with the SQL language

In [ ]:
pip install pandas

In [8]:
import pandas as pd

data = [
    (1, 2., 'Jakub'),
    (2, 3., 'Michał'),
    (3, 4., 'Aga')
]

In [9]:
df = pd.DataFrame(data, columns=['id', 'number', 'name'])

In [ ]:
df

In [ ]:
data = ((a, a*a) for a in range(1,10))

pd.DataFrame(data, columns=['a', 'a_square'])

#### Creating a RDD from a file

In [14]:
rdd = sc.parallelize([
    (1, 2., 'Jakub'),
    (2, 3., 'Michał'),
    (3, 4., 'Aga')
])


In [ ]:
rdd.collect()

In [16]:
df = spark.createDataFrame(rdd, schema=['a', 'b', 'name'])

In [ ]:
df

In [ ]:
df.printSchema()

In [ ]:
df.show()

In [ ]:
df.toPandas()

In [ ]:
!mkdir -p var
!wget https://wolnelektury.pl/media/book/txt/krzyzacy-tom-pierwszy.txt -O var/krzyzacy-1.txt
!wget https://wolnelektury.pl/media/book/txt/krzyzacy-tom-drugi.txt -O var/krzyzacy-2.txt

In [24]:
file = sc.textFile('var/krzyzacy*')

In [27]:
words = (
  file
    .filter(lambda x: "" != x)
    .flatMap(lambda x: x.split(" "))
    .map(lambda x: x.lower())
    .filter(lambda x: x != '—')
    .map(lambda x: (x,))
)

In [28]:
df = spark.createDataFrame(words, schema=['word'])

In [ ]:
df.show()

In [30]:
X = spark.createDataFrame([("Krakow", "1", {"foo": "boo"}), ("Warszawa", "2", {})], ['City', "digit", "attr"])

In [ ]:
X.show()

## Read from MULTIPLE sources

https://spark.apache.org/docs/latest/sql-data-sources.html

In [36]:
!echo '{"city":"Lublin","digit":5,"attr":{"foo":"zoo"}}' > cities.list
!echo '{"city":"Bielski","digit":2,"attr":{"sigma":"gamma"}}' >> cities.list

In [ ]:
cat cities.list

In [38]:
c = (spark.read
     .option('dropFieldIfAllNull', True)
     .option("primitivesAsString", True)
     .json("cities.list")
    )

In [ ]:
c.printSchema()

In [ ]:
c.show()

### functions
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html

In [42]:
import pyspark.sql.functions as F

In [43]:
multiplicate = F.udf(lambda x: int(x)*2)

In [44]:
c = c.withColumn("multipled", multiplicate(F.col("digit")))
c = c.withColumnRenamed('digit', 'number')

In [ ]:
c.show()

In [ ]:
(c
 .select(['attr.foo','city'])
 .where('(attr.foo is not null)')
).show()

In [47]:
T = spark.createDataFrame([
    ('nice one', ['foo', 'moo', 'boo', 'zoo']),
    ('bad one', ['foo', 'moo',])
],
['title', 'tags']
)

In [ ]:
T.select('title', F.explode('tags')).show()

In [53]:
kDF = spark.read.text('var/krzyzacy*.txt')


In [ ]:
kDF.show()

In [55]:
k = kDF.withColumn("NewCol", F.col("value"))

In [56]:
k = k.drop("value")

In [ ]:
k.show()

In [59]:
k = k.withColumns({'starts_with1': F.col("NewCol"), 'x': F.col("NewCol"), 'y': F.col('NewCol').startswith('A')})

In [ ]:
k.filter(k['y'] == True).show()

In [ ]:
!mkdir -p var
!wget -c https://data.edu.jkanclerz.com/data-science/uek/otomoto.csv -O var/otomoto.csv

In [65]:
raw_data = sc.textFile('var/otomoto.csv')
csv_data = (raw_data
            .map(lambda l: l.split(";"))
)


https://spark.apache.org/docs/latest/sql-data-sources-load-save-functions.html

In [76]:
DF = spark.read.load("var/otomoto.csv", format="csv", sep=";", inferSchema="true", header="true")

In [ ]:
DF.show()

In [ ]:
type(DF)

In [81]:
DF.createOrReplaceTempView("cars")

In [82]:
volvos = spark.sql("""
    Select `Marka pojazdu` as marka, `Model pojazdu` as model, `price`, `przebieg`, `Rok produkcji` as rok  from cars
    WHERE `Marka pojazdu` = "Volvo"
    AND `Model pojazdu` = "XC 60"
""")

In [ ]:
type(volvos)

The results of SQL queries are RDDs and support all the normal RDD operations.

In [ ]:
volvos.take(1)

In [ ]:
volvos.show()

In [ ]:
for volvo in volvos.collect():
    print("{} for {} with millage {}".format(volvo.model, volvo.price, volvo.przebieg))

In [ ]:
DF.printSchema()

### Queries as DataFrame operations


In [ ]:
(DF
     .select("Marka pojazdu")
     .groupBy("Marka pojazdu")
     .count()
     .show()
)

In [ ]:
DF.select(DF["Model pojazdu"]) \
    .filter(DF["Marka pojazdu"]=="Volvo") \
    .groupBy("Model pojazdu") \
    .count().show()

In [89]:
spark.stop()